In [1]:
# 这里引入datetime目的是为了借助isinstance(data,datetime.datetime)函数，挑选非时间格式行，进而清洗AQI文件中空行和重复标题行
import numpy as np
import pandas as pd
import datetime
import functools
import os

In [2]:
# debug装饰器
def debug(switch = True):
    def inner_debug(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            if switch:
                print(f'{datetime.datetime.now()} calling {func.__name__}...')
                return func(*args, **kwargs)
            else:
                return func(*args, **kwargs)
        return wrapper
    return inner_debug

In [3]:
# 清洗非时间序列的空行和重复标题行
@debug()
def clean_unnecessary (x):
    x.drop_duplicates(keep=False, inplace = True)
    x.replace('-', np.nan, inplace = True)
    x.set_index('日期', inplace = True)
    for i in x.columns[1:-3]:
        x[i].astype('float64')
    return x

In [4]:
# 替换站点名称
@debug()
def replace_oldname (x):
    old_to_new = {'复康路':'宾水西道',
                  '南京路':'大理道',
                  '航天路':'西四道',
                  '东环路':'北环路',
                  '广海道':'地纬路',
                  '新华道':'津同路',
                  '南口路':'中山北路',
                  '新华道':'津同路'}
    x.replace({'站点':old_to_new}, inplace = True)
    return x

In [5]:
# 时间处理
@debug()
def time_handler(x,y,start,end):
    # 获取年时间
    x_year_start = start
    x_year_end = (datetime.datetime.strptime(end,'%Y%m%d')-datetime.timedelta(1)).strftime('%Y%m%d')
    y_year_start = str(int(x_year_start[0:4])-1)+x_year_start[4:]
    y_year_end = str(int(x_year_end[0:4])-1)+x_year_end[4:]   
    # 获取月时间
    x_month_start = (datetime.datetime.strptime(end,'%Y%m%d')-datetime.timedelta(1)).replace(day=1).strftime('%Y%m%d')
    x_month_end = x_year_end
    y_month_start = str(int(x_month_start[0:4])-1)+x_month_start[4:]
    y_month_end = y_year_end
    # 获取周时间
    x_week_start = (datetime.datetime.strptime(end,'%Y%m%d')-datetime.timedelta(7)).strftime('%Y%m%d')
    x_week_end = x_year_end
    y_week_start = str(int(x_week_start[0:4])-1)+x_week_start[4:]
    y_week_end = y_year_end
    #  筛选时间
    x_year = x[x_year_start:x_year_end]
    y_year = y[y_year_start:y_year_end]
    x_month = x[x_month_start:x_month_end]
    y_month = y[y_month_start:y_month_end]
    x_week = x[x_week_start:x_week_end]
    y_week = y[y_week_start:y_week_end]
    
    return ([x_year,x_month,x_week],[y_year,y_month,y_week])

In [6]:
# 获取六项污染物数据、综合指数、达标天数、达标比率、重污染天数
@debug()
def get_data (x):
    # 获取原始六项污染物数据、综合指数、达标天数、达标比率、重污染天数
    y = x.groupby('站点').mean()[['SO2 浓度','NO2 浓度','PM10 浓度','PM2.5 浓度']]
    y['CO 浓度'] = x.groupby('站点').quantile(0.95)['CO 浓度']
    y['O3 8小时浓度'] = x.groupby('站点').quantile(0.9)['O3 8小时浓度']
    y['达'] = x[x['AQI'] <= 100].groupby('站点').count()['AQI']
    y['达'].fillna(0,inplace=True)
    y['达比'] = x[x['AQI'] <= 100].groupby('站点').count()['AQI']/x.groupby('站点').count()['AQI']
    y['重'] = x[x['AQI'] > 200 ].groupby('站点').count()['AQI']
    y['重'].fillna(0,inplace=True)
    # 获取修约后六项污染物数据、综合指数、达标天数、达标比率、重污染天数
    y['PM2.5'] = y['PM2.5 浓度'].round(10).round(0)
    y['PM10'] = y['PM10 浓度'].round(10).round(0)
    y['SO2'] = y['SO2 浓度'].round(10).round(0)
    y['NO2'] = y['NO2 浓度'].round(10).round(0)
    y['CO'] = y['CO 浓度'].round(10).round(1)
    y['O3'] = y['O3 8小时浓度'].round(10).round(0)
    y['达比'] = (y['达比']*100).round(1)
    y['综指未修'] =  y['PM2.5']/35 + y['PM10']/70 + y['SO2']/60 + y['NO2']/40 + y['CO']/4 + y['O3']/160
    y['综指'] =  ((((y['PM2.5']*100000).round(0)/35).round(-3)/100000).round(2) + (((y['PM10']*100000).round(0)/70).round(-3)/100000).round(2) + (((y['SO2']*100000).round(0)/60).round(-3)/100000).round(2) + (((y['NO2']*100000).round(0)/40).round(-3)/100000).round(2) + (((y['CO']*100000).round(0)/4).round(-3)/100000).round(2) + (((y['O3']*100000).round(0)/160).round(-3)/100000).round(2)).round(2)
    # 导出修约后结果
    y = y[['PM2.5','PM10', 'SO2', 'NO2','CO', 'O3','综指','达', '达比', '重','PM2.5 浓度','PM10 浓度','综指未修']]
    return y

In [7]:
# 合并今年和去年的求同比
@debug()
def year_to_year_growth(x,y):
    z = pd.merge(x,y,left_index=True,right_index=True)
    z['PM2.5同'] = round(round((z['PM2.5_x']-z['PM2.5_y'])*100/z['PM2.5_y'],10),1)
    z['PM10同'] = round(round((z['PM10_x']-z['PM10_y'])*100/z['PM10_y'],10),1)
    z['SO2同'] = round(round((z['SO2_x']-z['SO2_y'])*100/z['SO2_y'],10),1)
    z['NO2同'] = round(round((z['NO2_x']-z['NO2_y'])*100/z['NO2_y'],10),1)
    z['CO同'] = round(round((z['CO_x']-z['CO_y'])*100/z['CO_y'],10),1)
    z['O3同'] = round(round((z['O3_x']-z['O3_y'])*100/z['O3_y'],10),1)
    z['综指同'] = round(round((z['综指_x']-z['综指_y'])*100/z['综指_y'],10),1)
    z['达同'] = z['达_x']-z['达_y']
    z['达比同'] = (z['达比_x']-z['达比_y']).round(2)
    z['重同'] = z['重_x']-z['重_y']
    z['综指同未修'] = (z['综指_x']-z['综指_y'])*100/z['综指_y']
    z['综指未修同未修'] = (z['综指未修_x']-z['综指未修_y'])*100/z['综指未修_y']
    return pd.merge(x,z[['PM2.5同','PM10同','SO2同','NO2同','CO同','O3同','综指同',
                         '达同','达比同','重同','综指同未修','综指未修同未修']],left_index=True,right_index=True)

In [8]:
# 提取点位和各区数据并排序
@debug()
def extract_and_sort(x):
    # 定义替换名字词典
    position_to_district = {'宾水西道':'南开区',
                            '大理道':'和平区',
                            '勤俭道':'红桥区',
                            '北环路':'蓟州区',
                            '地纬路':'静海区',
                            '中山北路':'河北区',
                            '津沽路':'津南区',
                            '淮河道':'北辰区',
                            '渔阳路':'宝坻区',
                            '滨水东路':'宁河区',
                            '跃进路':'东丽区',
                            '国控点':'全市',
                            '第四大街':'滨海开发区',
                            '西四道':'滨海保税区',
                            '海泰发展二路':'滨海高新区',
                            '汉北路':'滨海生态城'}   
    position_to_position_14 = {'宾水西道':'南开宾水西道',
                               '大理道':'和平大理道',
                               '勤俭道':'红桥勤俭道',
                               '中山北路':'河北中山北路',
                               '津沽路':'津南津沽路',
                               '淮河道':'北辰淮河道',
                               '跃进路':'东丽跃进路',
                               '第四大街':'滨海第四大街',
                               '西四道':'滨海西四道',
                               '汉北路':'滨海汉北路',
                               '前进道':'河西前进道',
                               '大直沽八号路':'河东大直沽八号路',
                               '永明路':'滨海永明路',
                               '河西一经路':'滨海河西一经路'}  
    # 排序依据
    sortby = ['综排','PM2.5', 'PM10','SO2','NO2','CO','O3']
    # 提取点位和各区数据
    position = x.loc[['中山北路', '前进道', '勤俭道', '北环路', '团泊洼', '地纬路', '塘沽营口道',
       '大理道', '大直沽八号路', '天和路', '宝白公路', '宾水西道', '广安道', '永明路', '汉北路',
       '河西一经路', '津同路', '津沽路', '海泰发展二路', '淮河道', '渔阳路', '滨水东路',
       '第四大街', '西四道', '跃进路', '辛老路', '雍阳西道']]
    position_14 = x.loc[['中山北路', '前进道', '勤俭道','大理道', '大直沽八号路','宾水西道','永明路', '汉北路',
       '河西一经路','津沽路','淮河道','第四大街', '西四道', '跃进路']]
    district = x.loc[['中山北路', '勤俭道', '北环路','地纬路', '大理道', '宾水西道', '武清区','河东区', '河西区',
             '津沽路', '淮河道', '渔阳路', '滨水东路','滨海新区','西青区', '跃进路']]
    district_20 = x.loc[['中山北路', '勤俭道', '北环路','地纬路', '大理道', '宾水西道', '武清区','河东区', '河西区',
             '津沽路', '淮河道', '渔阳路', '滨水东路','滨海新区','西青区', '跃进路','西四道','海泰发展二路','第四大街','汉北路']]
    position_binhai = x.loc[['塘沽营口道','永明路', '汉北路','河西一经路', '海泰发展二路','第四大街', '西四道']]       
    #综合指数排序
    for i in [position, position_14, district, district_20, position_binhai]:
        #综合指数排名
        i.sort_values(by=['综指','综指未修'],inplace=True)
        i['综指排'] = i['综指'].rank(method = 'first')
        #综合指数改善率排名
        i.sort_values(by=['综指同未修','综指未修同未修'],inplace=True)
        i['综指同排'] = i['综指同未修'].rank(method = 'first')  
        #综合排名  
        i['综排'] = i['综指排']+i['综指同排']
        i.sort_values(by = sortby, inplace=True)
    #拼接并改名
    position = pd.concat([position,x.loc[['中心城区','环城四区','滨海新区','国控点']]],sort = False)
    
    position_14.rename(index = position_to_position_14,inplace=True)
   
    district = pd.concat([district,x.loc[['中心城区','国控点']]],sort = False).drop('中心城区')
    district.rename(index = position_to_district,inplace=True)
    
    district_20 = pd.concat([district_20,x.loc[['中心城区','国控点']]],sort = False).drop('中心城区')
    district_20.rename(index = position_to_district,inplace=True)
    
    position_binhai = pd.concat([position_binhai,x.loc[['滨海新区','国控点']]],sort = False)
    
    return position, position_14, district, district_20, position_binhai

In [9]:
# 导出文件
@debug()
def output(*args):
    # 参数设置
    columns_all=['SO2','SO2同','NO2','NO2同','PM10','PM10同','PM2.5','PM2.5同','CO','CO同','O3','O3同','达','达同','重','重同','达比','达比同','综指','综指同','综指排','综指同排','综排','综指未修','PM2.5 浓度','PM10 浓度','综指同未修','综指未修同未修']
    columns_short=['PM2.5','PM10','SO2','NO2','CO','O3','综指','综指同','PM2.5同','PM10同','综指排','综指同排','综排','综指未修','综指同未修','PM2.5 浓度','PM10 浓度','综指未修同未修','SO2同','NO2同','CO同','O3同','达','达同','重','重同','达比','达比同']
    path = r'D:\文件\Excel产品\计算结果'
    result_name = ['position.xlsx','position_14.xlsx', 'district.xlsx', 'district_20.xlsx', 'position_binhai.xlsx']
    period =['年', '月', '周']
    # 遍历循环导出
    for i,j in zip(zip(*args),result_name):
        writer = pd.ExcelWriter(path+'\\'+j)
        for k,l in zip(i,period):            
            k.to_excel(writer,sheet_name = l,columns=columns_short)
        writer.save()
    os.startfile(path)

In [10]:
# 主函数，定义路径，数据源、日期并导出数据
@debug()
def main(x, y, start = str(datetime.datetime.now().year)+'0101', end = datetime.datetime.now().strftime('%Y%m%d')):
    nums_of_variables = 3
    variables = list()
    # 读取数据
    x = pd.read_excel(x)
    y = pd.read_excel(y)
    print(f'读取数据时间为{datetime.datetime.now()-start_time}')
    # 清洗非时间序列的空行和重复标题行
    x = clean_unnecessary(x)
    y = clean_unnecessary(y)
    print('数据清洗正常！') if len(x)%35 == 0 else print('数据清洗异常！')
    # 替换站点名称
    x = replace_oldname(x)
    y = replace_oldname(y)
    # 筛选处理时间
    x_data, y_data = time_handler(x,y,start,end)
    # 获取六项污染物数据、综合指数、达标天数、达标比率、重污染天数
    for i in range(nums_of_variables):
        x_data[i] = get_data(x_data[i])
        y_data[i] = get_data(y_data[i])
    # 合并今年和去年并求同比
    for i in range(nums_of_variables):
        variables.append(year_to_year_growth(x_data[i], y_data[i]))
    # 求年、月、周的各项结果
    for i in range(nums_of_variables):
        variables[i] = extract_and_sort(variables[i])
    #导出文件
    output(*variables)

In [13]:
if __name__ == '__main__':
    start_time = datetime.datetime.now()
    main(x=r'D:\文件\Excel产品\AQI日报\2018.xlsx',y=r'D:\文件\Excel产品\AQI日报\2017.xlsx',start = '20180101',end = '20190101')
    print(f'总计花费时间为{datetime.datetime.now()-start_time}')

2019-02-14 09:34:37.324518 calling main...
读取数据时间为0:00:07.086405
2019-02-14 09:34:44.410923 calling clean_unnecessary...
2019-02-14 09:34:47.298088 calling clean_unnecessary...
数据清洗正常！
2019-02-14 09:34:47.383093 calling replace_oldname...
2019-02-14 09:34:47.390093 calling replace_oldname...
2019-02-14 09:34:47.398094 calling time_handler...
2019-02-14 09:34:48.080133 calling get_data...
2019-02-14 09:34:52.102363 calling get_data...
2019-02-14 09:34:52.360378 calling get_data...
2019-02-14 09:34:52.585390 calling get_data...
2019-02-14 09:34:52.690396 calling get_data...
2019-02-14 09:34:52.790402 calling get_data...
2019-02-14 09:34:52.892408 calling year_to_year_growth...
2019-02-14 09:34:52.903409 calling year_to_year_growth...
2019-02-14 09:34:52.913409 calling year_to_year_growth...
2019-02-14 09:34:52.924410 calling extract_and_sort...
2019-02-14 09:34:53.046417 calling extract_and_sort...
2019-02-14 09:34:53.093420 calling extract_and_sort...
2019-02-14 09:34:53.141422 calling 